# Quantum state tomography (QST) with LS, CS and CVX

In [8]:
# To call the libraries for the GD-QST
import sys
sys.path.insert(0, '..')
# You have to change the path of the library 
import os

from qutip import * 
from itertools import *
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt 
import qutip as qtp
#from qutip import basis, tensor

from qst_tec.least_square import least_square_qst
from qst_tec.compressed_sensing import compressed_sensing_qst
from qst_tec.convex_optimization_cvx import cvx_qst

import time


### First we will define the number of qubits and the measurement operators. We will use the Pauli matrices.


In [9]:
number_qubits: int = 3
dimension: int = 2**number_qubits
HS: int = dimension

# Creating the Measurement operators (Pauli matrices)

# computation basis, important for the other methods
def qubit_computation_basis(n):
    basis_states = [basis(2, i) for i in range(2)]
    return [tensor(*state) for state in product(basis_states, repeat=n)]

computation_basis = qubit_computation_basis(number_qubits)

operator_basis = [Qobj(tensor(computation_basis[i],computation_basis[j].dag()), 
                dims=([[2**number_qubits], [2**number_qubits]])) for i in range(len(computation_basis)) for j in range(len(computation_basis))]

pauli = [qeye(2), sigmax(), sigmay(), sigmaz()]
pauli_sys = [tensor(*op) for op in product(pauli, repeat=number_qubits)] # using itertools

Measu_ope = [Qobj(pauli_sys[i], dims=([[2**number_qubits], [2**number_qubits]])) for i in range(len(pauli_sys))]

print(Measu_ope[1:3])


#-----------------------------------------------------------------------
# It is necesary to transform the measurement operators from a numpy 
# type to a jax type (jax.np), that because of the jax grad 

ops_np = [op.full() for op in Measu_ope] # transforming from qutip object to numpy
ops_jnp = jnp.asarray(ops_np) # transforming from numpy to jax.np

[Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]], Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j]]]


## Implementing LS, CS, and CVX

For the case of a random density matrix for 3 qubits as the original density matrix (the one we want to reconstruct) created with QuTip $\rho_{random}$

In [10]:

max_itera: int = 300
batch_s = round(0.5*len(ops_jnp)) #len(ops_jnp)=4**N

# creating the original densirty matrix, this with a random rho from QuTip
rho_or = rand_dm(2**number_qubits)
data = qtp.expect(Measu_ope,rho_or)


#LS-QST
rho_ls, f_ls, t_ls = least_square_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, dimension=2**number_qubits, rho_ideal=rho_or)

#CS-QST
rho_cs, f_cs, t_cs = compressed_sensing_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, gamma=0, dimension=2**number_qubits, rho_ideal=rho_or)

rho_cvx, f_cvx, t_cvx = cvx_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, gamma=0, dimension=2**number_qubits, rho_ideal=rho_or)



In the cases of LS, CS, and CVX we just have a float value with the value of fidelity, the total time of reconstruction and the reconstructed density matrix. 

In [11]:
print('Final fidelity (LS) and average time in seconds :', f_ls, t_ls)
print('Final fidelity (CS) and average time in seconds :', f_cs, t_cs)
print('Final fidelity (CVX) and average time in seconds :', f_cvx, t_cvx)


Final fidelity (LS) and average time in seconds : 0.9999999986495418 0.028412818908691406
Final fidelity (CS) and average time in seconds : 0.9999999994062132 0.06327390670776367
Final fidelity (CVX) and average time in seconds : 0.9999999999998194 0.0380859375


In [12]:
print(rho_ls)

Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.15727637+0.j          0.0197209 -0.02519004j  0.03980829-0.0061778j
   0.00717159-0.0244281j  -0.02595271+0.07866587j -0.01402301-0.04464789j
   0.00899325-0.02922716j  0.01203279-0.06665354j]
 [ 0.0197209 +0.02519004j  0.15863635+0.j          0.00668831-0.04143068j
  -0.00793422+0.05959453j -0.02078111+0.01240021j -0.00221101-0.02269768j
  -0.01224415+0.02997325j -0.05255589+0.00869165j]
 [ 0.03980829+0.0061778j   0.00668831+0.04143068j  0.04743772+0.j
  -0.00341492-0.01160369j -0.01243138+0.0319273j   0.00383296-0.03921568j
  -0.02683753-0.02562274j -0.00554237-0.04422718j]
 [ 0.00717159+0.0244281j  -0.00793422-0.05959453j -0.00341492+0.01160369j
   0.07676134+0.j         -0.02820645+0.01734911j  0.02669118-0.02920314j
   0.01244106-0.0028239j   0.01404043+0.01528321j]
 [-0.02595271-0.07866587j -0.02078111-0.01240021j -0.01243138-0.0319273j
  -0.02820645-0.01734911j  0.17972553+0.j  